<a href="https://colab.research.google.com/github/balawillgetyou/dy/blob/master/dataPipeline20191201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automated data pipeline:
This notebook demonstrates an approach to grabbing multiple files of unstructured data from the web and extracting information them.

In [0]:
import requests
import re
import pandas as pd
import dateutil.parser as dparser

In [0]:
#obtaining 50 reports on livestock sales from a public data source
i = 0
x = 140
fileNames = []
while i < 49:
  target_url = 'https://www.ams.usda.gov/mnreports/rh_ls'+str(x)+'.txt'
  response = requests.get(target_url)
  if response:
    fileName = response.text[0:9]+str('.txt')
    fileName = fileName.replace('\r', '')
    fileNames.append(fileName)
    fileContent = response.text
    with open(fileName, "w") as file:  
      file.write(fileContent) 
    i += 1
    x += 1
  else:
    x += 1

In [0]:
#constructing a data table by extracting information from multiple unstructured text files 

cleaned_data = pd.DataFrame()
column_labels = ["category","head_sold"]
searchCondition = str("(\w+)\s+(\d+)\s+head")# this is how we get the number of livestock sold

#if the files were in a directory in the local machine, something like this would have to be used: for file in glob.glob(os.path.join(path, '*.txt')):
for file in fileNames:
    try:
      data = open(file).read()
   
      # find # of head sold
      sales = re.findall(searchCondition, data.lower())

      # name of market (always on fourth line)
      market = data.split(sep="\n")[3]

      # date reported
      date = dparser.parse(data[9:70],fuzzy=True).date()

      if len(sales) > 0:
        
        df_temp = pd.DataFrame.from_records(sales, columns=column_labels)
        df_temp["market"] = market
        df_temp["date_reported"] = date

      # add data about markets with no sheep sales
      else:
        df_temp = pd.DataFrame(data={"category": ["none"], "head_sold": [0]})
        df_temp["market"] = market
        df_temp["date_reported"] = date
    
      # append data to our cleaned data
      cleaned_data = cleaned_data.append(df_temp, ignore_index=True)
    except:
      pass

In [68]:
print(cleaned_data.shape)
print('-'*100)
print(cleaned_data.describe())
print('-'*100)
print(cleaned_data.head(5))
print('-'*100)

(253, 4)
----------------------------------------------------------------------------------------------------
       category head_sold                           market date_reported
count       253       253                              253           253
unique       26        94                               45            34
top       large         1  Marshall, VA Feeder Cattle Sale    2019-11-25
freq        107        64                               24            42
----------------------------------------------------------------------------------------------------
  category head_sold                                market date_reported
0   cattle       912  Northern Virginia Livestock Auctions    2019-11-25
1   steers       315  Northern Virginia Livestock Auctions    2019-11-25
2   steers         4  Northern Virginia Livestock Auctions    2019-11-25
3  heifers       442  Northern Virginia Livestock Auctions    2019-11-25
4    bulls       151  Northern Virginia Livestock Auctions 

Some ideas in this notebook were inspired by the work of Rachael Tatman on Kaggle. 